# Spark - RDD - Basic Transformations

In [1]:
#!pip install findspark

In [2]:
import findspark

findspark.init()

import pyspark

sc = pyspark.SparkContext(appName="RDDBasics")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/21 11:23:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc

<SparkContext master=local[*] appName=RDDBasics>

# Creando un RDD con 3 lineas de texto

* __Resilient__ - se va enviar los datos y las funciones que se quieren ejecutar a varios ordenadroes, pero cada trozito de datos es unico. Spark guardará los datos crudos y el planning de ejecución (incluye las funciones) y en caso de falla de una de los Workers se lo envia a otro.
* __Distributed__ - Separar los datos en Trabajadores (Workers). Un worker no es un slave, un worker es un proceso (PID) lo que quiere decir que en un "slave" pueden y normalmente hay mas de uno worker. Razón para poder usar todos los procesadores de la dicha maquina.
* __Dataset__ - Datos no estructurados - acepta cualquier tipo de datos.

In [4]:
rdd_lines = sc.parallelize(["linea 1 Python",
                            "linea 2 Python",
                            "linea 3 Spark"] )

In [5]:
type(rdd_lines)

pyspark.rdd.RDD

In [6]:
rdd_lines

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [7]:
rdd_lines.collect()

['linea 1 Python', 'linea 2 Python', 'linea 3 Spark']

## Ojo: rdd_lines.collect() es un "code smell"
Es decir tu codigo huele mal, porque estas trayendo 100% de los datos a Python, seguro que quieres hacer eso?

In [8]:
type(rdd_lines.collect())

list

In [9]:
rdd_numbers = sc.parallelize([1,2,3,3] )

In [10]:
rdd_numbers

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:274

.collect() returns the whole rdd()

In [11]:
rdd_lines.collect()

['linea 1 Python', 'linea 2 Python', 'linea 3 Spark']

In [12]:
rdd_numbers.collect() # Action that returns the whole RDD

[1, 2, 3, 3]

In [13]:
rdd_lines.take(10)

['linea 1 Python', 'linea 2 Python', 'linea 3 Spark']

In [14]:
rdd_numbers.take(10)

[1, 2, 3, 3]

In [15]:
sc.parallelize([1,2,3,3]).take(10)

[1, 2, 3, 3]

In [16]:
contador = rdd_numbers.count()
contador

4

In [17]:
contador

4

* Una __acción__ termina con el procesamiento en Spark y devuelve una respuesta al master con un tipo de datos de Python.
* Una __transformación__ no hace trigger (no dispara, no acciona) el procesamiento de Spark (es lazy evaluated),  lo quiere decir que si compruebo el tipo será algo de Spark no de Python.

## map vs. flatMap transformation
El flatMap reduce la dimensión de la lista y si es una lista de una dimensión, rompe la string.

In [18]:
rdd_lines.map(lambda x:x.split(" ")).take(4)

[['linea', '1', 'Python'], ['linea', '2', 'Python'], ['linea', '3', 'Spark']]

In [19]:
rdd_lines.flatMap(lambda x:x.split(" ")).take(10)

['linea', '1', 'Python', 'linea', '2', 'Python', 'linea', '3', 'Spark']

In [20]:
rdd_lines.map(lambda x: x.split(' ')).collect()

[['linea', '1', 'Python'], ['linea', '2', 'Python'], ['linea', '3', 'Spark']]

In [21]:
sc.parallelize(["linea 1 Python","linea 2 Python y Python","linea 3 Spark"]).map(lambda x:x.split(' ')). \
filter(lambda x: "Python" in x).count()

2

In [22]:
sc.parallelize(["linea 1 Python","linea 2 Python y Python","linea 3 Spark"]).flatMap(lambda x:x.split(' ')). \
filter(lambda x: "Python" in x).count()

3

In [23]:
rdd_lines.flatMap(lambda x: x.split(' ')).collect()

['linea', '1', 'Python', 'linea', '2', 'Python', 'linea', '3', 'Spark']

In [24]:
rdd_lines.map(lambda x:x).collect()

['linea 1 Python', 'linea 2 Python', 'linea 3 Spark']

In [25]:
rdd_lines.flatMap(lambda x:x).collect()

['l',
 'i',
 'n',
 'e',
 'a',
 ' ',
 '1',
 ' ',
 'P',
 'y',
 't',
 'h',
 'o',
 'n',
 'l',
 'i',
 'n',
 'e',
 'a',
 ' ',
 '2',
 ' ',
 'P',
 'y',
 't',
 'h',
 'o',
 'n',
 'l',
 'i',
 'n',
 'e',
 'a',
 ' ',
 '3',
 ' ',
 'S',
 'p',
 'a',
 'r',
 'k']

## filter transformation

In [26]:
for item in ["linea 1 Python","linea 2 Python","linea 3 Spark"]:
    print("Python" in item)

True
True
False


In [27]:
list(filter(lambda x: "Python" in x ,["linea 1 Python","linea 2 Python","linea 3 Spark"])) 
# Esto es Python, no Spark!

['linea 1 Python', 'linea 2 Python']

In [28]:
rdd_lines.filter(lambda x: "Python" in x).take(10)

['linea 1 Python', 'linea 2 Python']

# Juntando flatMap y filter

In [29]:
sc.parallelize(["linea 1 Python","linea 2 Python y Python y Python","linea 3 Spark"]). \
flatMap(lambda x:x.split(' ')). \
filter(lambda x: "Python" in x).take(10)

['Python', 'Python', 'Python', 'Python']

In [30]:
rdd_lines.flatMap(lambda x: x.split(' ')).filter(lambda x: "Python" in x).collect()

['Python', 'Python']

In [31]:
rdd_lines.filter(lambda x: "Python" in x).flatMap(lambda x: x.split(' ')).collect()

['linea', '1', 'Python', 'linea', '2', 'Python']

In [ ]:
#sc.p.map.filter.flatMap.map.......

### Cache
* Los caches en Spark se hacen en los workers (es en la memoria del PID) 
* Los caches son volatiles (cerrando Spark todo se pierden)
* Si pides un cache, es una proposión, si Spark no lo puede hacer, no lo hará!
* Si los caches no se utilizan en algun momento los borra
* Si los caches molestan a nuevos procesamiento los manda a disco duro

In [32]:
rdd_lines.cache()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

## distinct transformation

In [33]:
rdd_numbers = sc.parallelize([3,3,2,1] )
rdd_numbers.take(10)

[3, 3, 2, 1]

In [34]:
rdd_numbers.distinct().collect()

[1, 2, 3]

In [35]:
rdd_numbers.map(lambda x:x**2).filter(lambda x:x>3).distinct().count()#.take(10)

2

## sample without replacement

In [36]:
rdd_numbers.sample(False,0.5).collect()

[3]

## sample with replacement

In [37]:
rdd_numbers.sample(True,3).collect()

[3, 3, 3, 3, 3, 3, 2, 1, 1, 1, 1]

---

# Transformation SET OPERATIONS

In [38]:
rdd_numbers.collect()

[3, 3, 2, 1]

In [39]:
rdd_more_numbers = sc.parallelize([3,4,2,5])
rdd_more_numbers.collect()

[3, 4, 2, 5]

## union - como un append sin mas lista1.append(lista2)

In [40]:
rdd_numbers.union(rdd_more_numbers).collect()

[3, 3, 2, 1, 3, 4, 2, 5]

## intersection - INNER JOIN

In [41]:
rdd_numbers.intersection(rdd_more_numbers).collect()

[2, 3]

## subtraction - Left Outer join

In [ ]:
rdd_numbers.subtract(rdd_more_numbers).collect()

## cartesian product

In [ ]:
rdd_numbers.cartesian(rdd_more_numbers).collect()

In [ ]:
rdd_numbers.cartesian(rdd_more_numbers).map(lambda x: x[0]/x[1]).take(100)

In [ ]:
rdd_numbers.cartesian(rdd_more_numbers).map(lambda x:x[0]+x[1]).collect()

## Exercise 1) sum rdd1 + rdd2

Expected result:
['(1+3)=4',
 '(1+4)=5',
 '(2+3)=5',
 '(2+4)=6',
 '(1+2)=3',
 '(1+5)=6',
 '(2+2)=4',
 '(2+5)=7',
 '(3+3)=6',
 '(3+4)=7',
 '(3+3)=6',
 '(3+4)=7',
 '(3+2)=5',
 '(3+5)=8',
 '(3+2)=5',
 '(3+5)=8']


In [45]:
rdd1 = sc.parallelize([1,2,3,3])

In [46]:
rdd2 = sc.parallelize([3,4,2,5])

In [55]:
rdd1.cartesian(rdd2).map(lambda x: f'{x[0]}+{x[1]}={x[0] + x[1]}').collect()

['1+3=4',
 '1+4=5',
 '1+2=3',
 '1+5=6',
 '2+3=5',
 '2+4=6',
 '2+2=4',
 '2+5=7',
 '3+3=6',
 '3+4=7',
 '3+2=5',
 '3+5=8',
 '3+3=6',
 '3+4=7',
 '3+2=5',
 '3+5=8']

## Exercise 2) Explain what is the code doing, input (values and technology/type), technologies involved and output (values and technology/type)

rdd = sc.parallelize([1,2,3,4]) transformacion. devuelve

rdd.map(lambda x: x * 2) transformacion. Devuelve spark

.collect() accion. Devuelve python

## Exercise 3) What is wrong with the following code and how to fix it?

len(sc.parallelize([1,2,3,4]).map(lambda x: x * 2).collect())

In [56]:
# Que estas llevando la lista a local y destruyes la logica distribuida. La solucion sería:
sc.parallelize([1,2,3,4]).map(lambda x: x * 2).count()

4

---

# Spark - RDD - Basic Actions

## collect

In [57]:
rdd_numbers = sc.parallelize([1,2,3,3])

In [58]:
rdd_numbers.collect()

[1, 2, 3, 3]

## count

In [59]:
rdd_numbers.count()

4

## countByValue - same as value_counts() in DataFrame in Pandas

In [60]:
rdd_more_numbers = sc.parallelize([4,5,2,7])

In [61]:
rdd_many_numbers = rdd_numbers.union(rdd_more_numbers)

In [62]:
rdd_many_numbers.collect()

[1, 2, 3, 3, 4, 5, 2, 7]

In [63]:
rdd_many_numbers.countByValue()

defaultdict(int, {1: 1, 2: 2, 3: 2, 4: 1, 5: 1, 7: 1})

In [65]:
#df.value_counts()

In [66]:
rdd_lines.flatMap(lambda x:x.split(" ")).countByValue()#take(10)

defaultdict(int, {'linea': 3, '1': 1, 'Python': 2, '2': 1, '3': 1, 'Spark': 1})

## Ejercicio 4) calcular el numero de ocurrencia de cada palabras en rdd_lines

In [67]:
rdd_lines = sc.parallelize(["linea 1 Python","linea 2 Python","linea 3 Spark"] )

In [68]:
rdd_lines.flatMap(lambda x: x.split(' ')).countByValue()#.take(10) 

defaultdict(int, {'linea': 3, '1': 1, 'Python': 2, '2': 1, '3': 1, 'Spark': 1})

### Transformaciones 0 o más? Porque el resultado de una transformación de un objeto de Spark siempre es el mismo objeto, en el caso actual RDD en otro RDD.
### Acciones - 0 o 1? Porque una acción devulve la respuesta a Python en un objeto de Python, ya no hay nade de Spark funcionando!

## take - same as head() in DataFrame in Pandas

In [ ]:
rdd_many_numbers.take(2)

## top - return the highest values

In [ ]:
rdd_more_numbers = sc.parallelize([3,4,5,2,5])

In [ ]:
rdd_more_numbers.top(3)

In [ ]:
rdd_more_numbers.take(3)

### Ejercicio 5) coger los 3 valores unicos máximos.

In [ ]:
rdd_more_numbers.distinct().top(3)

In [ ]:
rdd_lines.distinct().take(3)

In [ ]:
rdd_lines.distinct().top(2)

## takeOrdered

In [ ]:
rdd_more_numbers.collect()

In [ ]:
rdd_more_numbers.take(3)

In [ ]:
rdd_more_numbers.takeOrdered(3,lambda x: -x) # Descending

In [ ]:
rdd_more_numbers.takeOrdered(3,lambda x: x) # Ascending

In [ ]:
rdd_more_numbers.takeOrdered(3) # Ascending

### Ejercicio 6) coger 3 valores por orden, los pares primero en orden descreciente y los impares en orden cresciente.

In [71]:
sc.parallelize([5,2,5,4,3]).takeOrdered(10, lambda x: x%2)

[2, 4, 5, 5, 3]

### Ejercicio 7) coger 3 valores por orden, los pares primero en orden descreciente y los impares a continuación en orden tambien decresciente.

In [ ]:
sc.parallelize([5,2,5,4,3]).takeOrdered(10, lambda x: x%2)

---

# Persist

In [ ]:
print(dir())

In [ ]:
a = [1,2,3,3] # En casi todas las lenguajes de programación las variables son persistentes dentro de su ambito!

In [ ]:
print(dir())

In [ ]:
rdd_numbers = sc.parallelize([1,2,3,3]) 
# Un RDD de Spark es volatil!!!! Solo existe cuando hay una acción y luego desaparece

In [ ]:
rdd_more_numbers.persist

In [ ]:
rdd_more_numbers.takeOrdered(rdd_more_numbers.count()) # Ascending

In [ ]:
rdd_more_numbers.unpersist

---

In [ ]:
rdd_more_numbers.takeOrdered(3,lambda x: -x**4 if x % 2 == 0 else -x) # Ascending

## takeSample

In [ ]:
rdd_many_numbers.collect()

In [ ]:
rdd_many_numbers.count()

In [ ]:
rdd_many_numbers.take(10)

In [ ]:
rdd_many_numbers.takeSample(False,20,seed=321) #Without replacement

In [ ]:
rdd_many_numbers.takeSample(True,20,seed=321) #With replacement

In [ ]:
for semilla in range(20):
    print(rdd_many_numbers.takeSample(True,10,seed=semilla)) #With replacement

### Ejercicio 8) crear 10 listas aleatorias con 10 elementos de rdd_many_numbers usando semillas diferentes y unirlas todas en un unico rdd. ojo: se puede hacer persistent alguna de ellas?

---

# Spark - RDD - Reduce Actions - Reducing the whole list to a single value

## reduce

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


SELECT COUNT(*) FROM tabla1 - la acción COUNT - es un agregador en SQL, en Python/Spark llamamos de reductor o reduce

In [ ]:
rdd = sc.parallelize([1, 2, 3, 4]) 
rdd.reduce(lambda a, b: a + b)

In [ ]:
(1*2)*(3*4)

In [ ]:
rdd_many_numbers.collect()

In [ ]:
rdd_many_numbers.reduce(lambda a, b: a * b)

## fold - the same as reduce, but you can provide a starting value

In [ ]:
sc.parallelize([1,25,8,4,2]).fold(0,lambda a,b:a+b)

In [ ]:
1+25+8+4+2

In [ ]:
sc.parallelize([1,25,8,4,2]).fold(1,lambda a,b:a+b)

In [ ]:
min([1,2,3,4,5])

In [ ]:
max([1,2,3,4,5])

## aggregate

In [ ]:
sc.parallelize([1,2,3,4,5]).reduce(lambda a,b:(a+b)/2)

In [ ]:
# sc.parallelize([1,2,3,4,5]).aggregate(INICIALIZADORES,REDUCE_DENTRO_WORKER, REDUCE_ENTRE_WORKERS)

# sc.parallelize([1,2,3,4,5]).aggregate((0),lambda acumlulador,valor_de_linea:x, lambda x,y:x)
t = sc.parallelize([1,2,3,4,5]).aggregate((0,0,1), \
                                      lambda x,y:(x[0]+y,x[1]+1,x[2]*y), \
                                      lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]*y[2]))
# x es el acumulador
# y son los valores


In [ ]:
t[0]/t[1]

In [ ]:
t[2]**(1/t[1])

In [ ]:
t = sc.parallelize([1,2,3,4,5]).aggregate(
  (0, 0), # INICIALIZADOR DE LOS DOS CONTADORES 
  (lambda acc, value: (acc[0] + value, acc[1] + 1)), # REDUCE dentro del WORKER
  (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))) #REDUCE entre WORKERS

In [ ]:
t

In [ ]:
t[0]/t[1]

## reduce by Key

In [ ]:
[(1,2), (3,4), (3,6)]

In [ ]:
d = {1:2, 3:4,3:6}
d

In [ ]:
rdd = sc.parallelize({1:2, 3:4,3:6})
rdd.collect()

In [ ]:
rdd.reduceByKey(lambda a, b: a + b).collect()

In [ ]:
[("Python",20),("Python",50),("Python",35),("Spark",23)]

In [ ]:
sc.parallelize([(None,30),(4+4j,50),(4+4j,35),(4+4j,23)]).\
reduceByKey(lambda x,y: x+y).\
collect()

In [ ]:
sc.parallelize([("Python",20),("Python",50),("Python",35),("Spark",23)]).\
reduceByKey(lambda x,y: x+y).\
collect()

In [ ]:
[("Python",(20,3)),("Python",(50,4)),("Python",(35,2)),("Spark",(20,3))]

In [ ]:
sc.parallelize([("Python",(20,3)),("Python",(50,4)),("Python",(35,2)),("Spark",(20,3))]).\
reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).\
collect()

## Persistent (Catching)

In [ ]:
rdd.persist

In [ ]:
rdd.count()

## Cache

In [ ]:
rdd_cached_lines = rdd_lines.cache()

In [ ]:
rdd_cached_lines.collect()

In [ ]:
rdd_cached_lines.count()

---

# Example 1

In [ ]:
lines = sc.parallelize(["linea 500 Python","linea 404 Python","linea 404 Spark"] )

In [ ]:
lines.map(lambda x: x.split(' ')).filter(lambda x : "404" in x).map(lambda word : (word, 1)).collect()

In [ ]:
lines.flatMap(lambda x: x.split(' ')).collect()

In [ ]:
lines.map(lambda x: x.split(' ')).collect()

In [ ]:
lines.flatMap(lambda x: x.split(' ')).filter(lambda x : "404" in x).collect()

In [ ]:
lines.flatMap(lambda x: x.split(' ')).\
filter(lambda x : "404" in x).\
map(lambda word : (word, 1)).collect()

In [ ]:
from operator import add
#lambda x,y: x+y

In [ ]:
lines.flatMap(lambda x: x.split(' ')).\
filter(lambda x : "404" in x).\
map(lambda word : (word, 1)).\
reduceByKey(add).collect()

In [ ]:
lines.flatMap(lambda x: x.split(' ')).filter(lambda x : "404" in x or "500" in x).count()

In [ ]:
from operator import add

In [ ]:
rdd = lines.flatMap(lambda x: x.split(' '))
rdd.map(lambda x:(x,1)).reduceByKey(add).collect()

In [ ]:
lines.flatMap(lambda x: x.split(' '))\
.filter(lambda x : "404" in x or "500" in x).map(lambda word : (word, 1)) \
.reduceByKey(add).collect()

In [ ]:
lines.flatMap(lambda x: x.split(' ')).filter(lambda x : "404" in x or "500" in x).map(lambda word : (word, 1)) \
.reduceByKey(lambda x,y: x+y).collect()

# Ejercicio - Usando %3, sumar los número entre 1 y 1000000 divisible por %3==0, %3==1 o %3==2 (como si fuera por grupo). Hasta las 11:00
respuesta esperada: (166666833333, 166667166667, 166666500000)

# Ejercicio 2) Hacer el anterior juntando las claves 0 y 1
respuesta esperada: (333334000000, 166666500000)